In [ ]:
import random         # 导入 random 模块，用于随机数生成
import torch          # 导入 PyTorch 模块，用于深度学习任务
import numpy as np    # 导入 numpy 模块，用于数值计算
from torch import nn  # 从 PyTorch 中导入神经网络模块
from sklearn import datasets  # 从sklearn引入数据集
from sklearn.model_selection import train_test_split  # 导入 sklearn 库中的 train_test_split 函数，用于数据划分
from sklearn.preprocessing import StandardScaler     # 导入 sklearn 库中的 StandardScaler 类，用于数据标准化

from torch.optim.lr_scheduler import StepLR
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "5"
#from sklearn.datasets import load_wine
#"""
import rpy2.robjects as robjects
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
#"""
from torch.utils.data import Dataset,DataLoader

import joblib
import math
imagenet_max=0
imagenet_min=0
imagenet_std=0
imagenet_mean=0
imagenet_max_sd=0
imagenet_min_sd=0
imagenet_std_sd=0
imagenet_mean_sd=0
imagenet_max_nat=0
imagenet_min_nat=0
imagenet_std_nat=0
imagenet_mean_nat=0
imagenet_max_wga=0
imagenet_min_wga=0
imagenet_std_wga=0
imagenet_mean_wga=0
train_list={}
test_list={}
def normalize_train(x,sd=False,nat=False,wga=False):
    if sd:
        global imagenet_mean_sd
        imagenet_mean_sd = np.mean(x)
        global imagenet_std_sd
        imagenet_std_sd = np.std(x) 
        x=(x - imagenet_mean_sd) / (imagenet_std_sd * 1.0)
        global imagenet_max_sd
        imagenet_max_sd=np.max(x)
        global imagenet_min_sd
        imagenet_min_sd=np.min(x)
        print(imagenet_max_sd,imagenet_min_sd,imagenet_std_sd,imagenet_mean_sd)
        return (x-imagenet_min_sd)/(imagenet_max_sd-imagenet_min_sd)
    elif nat:
        global imagenet_mean_nat
        imagenet_mean_nat = np.mean(x)
        global imagenet_std_nat
        imagenet_std_nat = np.std(x) 
        x=(x - imagenet_mean_nat) / (imagenet_std_nat * 1.0)
        global imagenet_max_nat
        imagenet_max_nat=np.max(x)
        global imagenet_min_nat
        imagenet_min_nat=np.min(x)
        print(imagenet_max_nat,imagenet_min_nat,imagenet_std_nat,imagenet_mean_nat)
        return (x-imagenet_min_nat)/(imagenet_max_nat-imagenet_min_nat)
    elif wga:
        global imagenet_mean_wga
        imagenet_mean_wga = np.mean(x)
        global imagenet_std_wga
        imagenet_std_wga = np.std(x) 
        x=(x - imagenet_mean_wga) / (imagenet_std_wga * 1.0)
        global imagenet_max_wga
        imagenet_max_wga=np.max(x)
        global imagenet_min_wga
        imagenet_min_wga=np.min(x)
        print(imagenet_max_wga,imagenet_min_wga,imagenet_std_wga,imagenet_mean_wga)
        return (x-imagenet_min_wga)/(imagenet_max_wga-imagenet_min_wga)
    else:
        global imagenet_mean
        imagenet_mean = np.mean(x) 
        global imagenet_std
        imagenet_std = np.std(x) 
        x=(x - imagenet_mean) / (imagenet_std * 1.0)
        global imagenet_max
        imagenet_max=np.max(x)
        global imagenet_min
        imagenet_min=np.min(x)
        print(imagenet_max,imagenet_min,imagenet_std,imagenet_mean)
        return (x-imagenet_min)/(imagenet_max-imagenet_min) 

def normalize_test(x,sd=False,nat=False,wga=False):
    if sd:
        x=(x - imagenet_mean_sd) / (imagenet_std_sd * 1.0)
        print(imagenet_max_sd,imagenet_min_sd,imagenet_std_sd,imagenet_mean_sd)
        return (x-imagenet_min_sd)/(imagenet_max_sd-imagenet_min_sd)
    elif nat:
        x=(x - imagenet_mean_nat) / (imagenet_std_nat * 1.0)
        print(imagenet_max_nat,imagenet_min_nat,imagenet_std_nat,imagenet_mean_nat)
        return (x-imagenet_min_nat)/(imagenet_max_nat-imagenet_min_nat)
    elif wga:
        x=(x - imagenet_mean_wga) / (imagenet_std_wga * 1.0)
        print(imagenet_max_wga,imagenet_min_wga,imagenet_std_wga,imagenet_mean_wga)
        return (x-imagenet_min_wga)/(imagenet_max_wga-imagenet_min_wga)
    else:
        x=(x - imagenet_mean) / (imagenet_std * 1.0)
        print(imagenet_max,imagenet_min,imagenet_std,imagenet_mean)
        return (x-imagenet_min)/(imagenet_max-imagenet_min)

class origin_dataset_train(Dataset):
    def __init__(self, transform,valid):
        super(origin_dataset_train, self).__init__()
        #"""
        path_train = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_time_12_code_filter_iso_offset.rds'
        path_train_base_num_fraction = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_base_num_fraction_time_12_code_filter_iso_offset.rds'
        path_train_std_diff = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_std_diff_time_12_code_filter_iso_offset.rds'
        path_train_time_diff = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_time_diff_time_12_code_filter_iso_offset.rds'
        path_train_basic_group = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_basic_group_time_12_code_filter_iso_offset.rds'
        path_train_t_test_pval = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_t_test_pval_time_12_code_filter_iso_offset.rds'
        path_train_u_test_pval = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_u_test_pval_time_12_code_filter_iso_offset.rds'
        path_train_avg_basequality = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_avg_basequality_time_12_code_filter_iso_offset.rds'
        path_train_g_num = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_g_num_time_12_code_filter_iso_offset.rds'
        path_train_g_value = '/home/nipeng/chenzheng/data.basic_group/All46_train_data_balance_g_value_time_12_code_filter_iso_offset.rds'

        readRDS = robjects.r['readRDS']
        file_train = readRDS(path_train)
        file_train_base_num_fraction  = readRDS(path_train_base_num_fraction)
        file_train_std_diff  = readRDS(path_train_std_diff)
        file_train_time_diff  = readRDS(path_train_time_diff)
        file_train_basic_group  = readRDS(path_train_basic_group)
        file_train_t_test_pval  = readRDS(path_train_t_test_pval)
        file_train_u_test_pval  = readRDS(path_train_u_test_pval)
        file_train_avg_basequality  = readRDS(path_train_avg_basequality)
        file_train_g_num  = readRDS(path_train_g_num)
        file_train_g_value  = readRDS(path_train_g_value)

        with localconverter(ro.default_converter + pandas2ri.converter):
            pd_from_r_df_train = ro.conversion.rpy2py(file_train)
            pd_from_r_df_train_base_num_fraction = ro.conversion.rpy2py(file_train_base_num_fraction)
            pd_from_r_df_train_std_diff = ro.conversion.rpy2py(file_train_std_diff)
            pd_from_r_df_train_time_diff = ro.conversion.rpy2py(file_train_time_diff)
            pd_from_r_df_train_basic_group = ro.conversion.rpy2py(file_train_basic_group)
            pd_from_r_df_train_t_test_pval = ro.conversion.rpy2py(file_train_t_test_pval)
            pd_from_r_df_train_u_test_pval = ro.conversion.rpy2py(file_train_u_test_pval)
            pd_from_r_df_train_avg_basequality = ro.conversion.rpy2py(file_train_avg_basequality)
            pd_from_r_df_train_g_num = ro.conversion.rpy2py(file_train_g_num)
            pd_from_r_df_train_g_value = ro.conversion.rpy2py(file_train_g_value)

        """
        #motifs = ["GGW5mCC", "GA6mANNNNNNTRG", "T4mCTTC", "4mCTNAG", "GT6mANNNNNCTC", "RG5mCGCY"]
        #motifs = ["C5mCGCGG", "G5mCCGGC", "G6mAGNNNNNTAC", "GC6mANNNNNNNNTGC", "GG5mCC", "GGNN5mCC","GGTG6mA","GT6mANNNNNCTC","RG5mCGCY"] #NG
        #motifs = ["4mCTNAG", "AG4mCT", "CCA4mCGK", "G6mATC", "GCYYG6mAT","GTA4mC"] #MH
        motifs = ["4mCCGG", "ATTA6mAT", "C6mATG", "CRT6mANNNNNNNWC", "CS6mAG","CTRY6mAG", "CY6mANNNNNNTTC", "G5mCGC","G6mAGG"] #HP
        mask = pd_from_r_df_train["motif"].isin(motifs)    
        pd_from_r_df_train=pd_from_r_df_train[~mask]
        pd_from_r_df_train_base_num_fraction=pd_from_r_df_train_base_num_fraction[~mask]
        pd_from_r_df_train_std_diff=pd_from_r_df_train_std_diff[~mask]
        pd_from_r_df_train_time_diff=pd_from_r_df_train_time_diff[~mask]
        pd_from_r_df_train_basic_group=pd_from_r_df_train_basic_group[~mask]
        pd_from_r_df_train_t_test_pval=pd_from_r_df_train_t_test_pval[~mask]
        pd_from_r_df_train_u_test_pval=pd_from_r_df_train_u_test_pval[~mask]
        pd_from_r_df_train_avg_basequality=pd_from_r_df_train_avg_basequality[~mask]
        pd_from_r_df_train_g_num=pd_from_r_df_train_g_num[~mask]
        pd_from_r_df_train_g_value=pd_from_r_df_train_g_value[~mask]
        """
        self.data=pd_from_r_df_train.values[:,2:]
        self.data = self.data.astype(float) 
        self.data_base_num_fraction=pd_from_r_df_train_base_num_fraction.values[:,2:]
        self.data_base_num_fraction = self.data_base_num_fraction.astype(float) 
        self.data_std_diff=pd_from_r_df_train_std_diff.values[:,2:]
        self.data_std_diff = self.data_std_diff.astype(float) 
        self.data_time_diff=pd_from_r_df_train_time_diff.values[:,2:]
        self.data_time_diff = self.data_time_diff.astype(float) 
        self.data_basic_group=pd_from_r_df_train_basic_group.values[:,2:]
        self.data_basic_group_str = [''.join(basic_group) for basic_group in self.data_basic_group]
        #print(self.data_basic_group_str[:10])
        self.data_t_test_pval=pd_from_r_df_train_t_test_pval.values[:,2:]
        self.data_t_test_pval = self.data_t_test_pval.astype(float) 
        self.data_u_test_pval=pd_from_r_df_train_u_test_pval.values[:,2:]
        self.data_u_test_pval = self.data_u_test_pval.astype(float) 
        self.data_avg_basequality=pd_from_r_df_train_avg_basequality.values[:,2:]
        self.data_avg_basequality = self.data_avg_basequality.astype(float) 
        self.data_g_num=pd_from_r_df_train_g_num.values[:,2:]
        self.data_g_num = self.data_g_num.astype(float) 
        self.data_g_value=pd_from_r_df_train_g_value.values[:,2:]
        self.data_g_value = self.data_g_value.astype(float) 
        """
        self.data_sd_diff=pd_from_r_df_train_sd_diff.values[:,2:]
        self.data_sd_diff = self.data_sd_diff.astype(float) 
        self.data_sd_nat=pd_from_r_df_train_sd_nat.values[:,2:]
        self.data_sd_nat = self.data_sd_nat.astype(float) 
        self.data_sd_wga=pd_from_r_df_train_sd_wga.values[:,2:]
        self.data_sd_wga = self.data_sd_wga.astype(float) 
        self.data_mean_nat=pd_from_r_df_train_mean_nat.values[:,2:]
        self.data_mean_nat = self.data_mean_nat.astype(float) 
        self.data_mean_wga=pd_from_r_df_train_mean_wga.values[:,2:]
        self.data_mean_wga = self.data_mean_wga.astype(float) 

        self.data_basic_group=pd_from_r_df_train_basic_group.values[:,2:]
        #embedding
        self.data_basic_group[self.data_basic_group=='A']=0
        self.data_basic_group[self.data_basic_group=='C']=1
        self.data_basic_group[self.data_basic_group=='T']=2
        self.data_basic_group[self.data_basic_group=='G']=3
        
        self.data_basic_group = self.data_basic_group.astype(int) 
        """
        #self.data_basic_group=np.eye(4)[self.data_basic_group]


        #len_train=int(0.8*len(self.data))
        y_train=pd_from_r_df_train.values[:,:1]            
        y_train=np.squeeze(y_train)
        y_train_split=[i.split("_") for i in y_train]
        y_train_split=np.array(y_train_split)
        y_train_basic_group=y_train_split[:,0]
        y_train_offset=y_train_split[:,1]

        y_train_basic_group[y_train_basic_group=='5mC']=0 
        y_train_basic_group[y_train_basic_group=='4mC']=1
        y_train_basic_group[y_train_basic_group=='6mA']=2
        y_train_offset[y_train_offset=='-12']=0 
        y_train_offset[y_train_offset=='-11']=1
        y_train_offset[y_train_offset=='-10']=2
        y_train_offset[y_train_offset=='-9']=3
        y_train_offset[y_train_offset=='-8']=4
        y_train_offset[y_train_offset=='-7']=5
        y_train_offset[y_train_offset=='-6']=6   
        y_train_basic_group=np.array(y_train_basic_group,dtype=np.int64)   
        y_train_offset=np.array(y_train_offset,dtype=np.int64)
        """
        joblib.dump(self.data,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_mean_12_code_filter_iso_wo_MH.pkl") 
        joblib.dump(self.data_base_num_fraction,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_base_num_fraction_12_code_filter_iso_wo_MH.pkl") 
        joblib.dump(y_train_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_basic_group_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(y_train_offset,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_offset_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_std_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_std_diff_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_time_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_time_diff_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_basic_group_str,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_basic_group_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_t_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_t_test_pval_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_u_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_u_test_pval_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_avg_basequality,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_avg_basequality_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_g_num,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_g_num_12_code_filter_iso_wo_MH.pkl")
        joblib.dump(self.data_g_value,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_g_value_12_code_filter_iso_wo_MH.pkl")
        """
        print("kkk")
        """
        joblib.dump(self.data_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_basic_valid_1_290.pkl")
        joblib.dump(self.data_sd_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_sd_valid_1_290.pkl")

        joblib.dump(self.data_mean_nat,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_mean_nat_valid_1_290.pkl")
        joblib.dump(self.data_mean_wga,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_mean_wga_valid_1_290.pkl")
        joblib.dump(self.data_sd_nat,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_sd_nat_valid_1_290.pkl")
        joblib.dump(self.data_sd_wga,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_sd_wga_valid_1_290.pkl")
        """
        #"""
        if valid==False:
            all_list=set(range(len(self.data)))
            global train_list
            train_list=set(np.random.choice(range(len(self.data)), int(len(self.data)*0.95), replace=False))
            global test_list
            test_list=all_list-train_list
            print(np.array(list(train_list)).shape)
            print(np.array(self.data_basic_group_str))
            self.data=self.data.take(np.array(list(train_list)),0)
            self.data_base_num_fraction=self.data_base_num_fraction.take(np.array(list(train_list)),0)
            self.data_std_diff=self.data_std_diff.take(np.array(list(train_list)),0)
            self.data_time_diff=self.data_time_diff.take(np.array(list(train_list)),0)
            self.data_t_test_pval=self.data_t_test_pval.take(np.array(list(train_list)),0)
            self.data_u_test_pval=self.data_u_test_pval.take(np.array(list(train_list)),0)
            y_train_basic_group=y_train_basic_group.take(np.array(list(train_list)),0) 
            y_train_offset=y_train_offset.take(np.array(list(train_list)),0) 
            self.data_basic_group_str=np.array(self.data_basic_group_str).take(np.array(list(train_list)),0) 
            self.data_avg_basequality=self.data_avg_basequality.take(np.array(list(train_list)),0)
            self.data_g_num=self.data_g_num.take(np.array(list(train_list)),0)
            self.data_g_value=self.data_g_value.take(np.array(list(train_list)),0)
            #y_train_basic_group=np.array(y_train_basic_group,dtype=np.int64)   
            #y_train_offset=np.array(y_train_offset,dtype=np.int64)  
            joblib.dump(self.data,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_mean_12_code_filter_iso_offset_train.pkl") 
            joblib.dump(self.data_base_num_fraction,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_base_num_fraction_12_code_filter_iso_offset_train.pkl") 
            joblib.dump(y_train_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_basic_group_12_code_filter_iso_offset_train.pkl")
            joblib.dump(y_train_offset,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_offset_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_std_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_std_diff_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_time_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_time_diff_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_basic_group_str,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_basic_group_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_t_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_t_test_pval_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_u_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_u_test_pval_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_avg_basequality,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_avg_basequality_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_g_num,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_g_num_12_code_filter_iso_offset_upsample_train.pkl")
            joblib.dump(self.data_g_value,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_g_value_12_code_filter_iso_offset_upsample_train.pkl")
        else:
            self.data=self.data.take(np.array(list(test_list)),0)
            self.data_base_num_fraction=self.data_base_num_fraction.take(np.array(list(test_list)),0)
            self.data_std_diff=self.data_std_diff.take(np.array(list(test_list)),0)
            self.data_time_diff=self.data_time_diff.take(np.array(list(test_list)),0)
            self.data_t_test_pval=self.data_t_test_pval.take(np.array(list(test_list)),0)
            self.data_u_test_pval=self.data_u_test_pval.take(np.array(list(test_list)),0)
            y_train_basic_group=y_train_basic_group.take(np.array(list(test_list)),0) 
            y_train_offset=y_train_offset.take(np.array(list(test_list)),0) 
            self.data_basic_group_str=np.array(self.data_basic_group_str).take(np.array(list(test_list)),0) 
            self.data_avg_basequality=self.data_avg_basequality.take(np.array(list(test_list)),0)
            self.data_g_num=self.data_g_num.take(np.array(list(test_list)),0)
            self.data_g_value=self.data_g_value.take(np.array(list(test_list)),0)
            #y_train_basic_group=np.array(y_train_basic_group,dtype=np.int64)   
            #y_train_offset=np.array(y_train_offset,dtype=np.int64)  
            joblib.dump(self.data,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_mean_12_code_filter_iso_offset_upsample_valid.pkl") 
            joblib.dump(self.data_base_num_fraction,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_base_num_fraction_12_code_filter_iso_offset_upsample_valid.pkl") 
            joblib.dump(y_train_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_basic_group_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(y_train_offset,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_offset_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_std_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_std_diff_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_time_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_time_diff_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_basic_group_str,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_basic_group_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_t_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_t_test_pval_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_u_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_u_test_pval_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_avg_basequality,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_avg_basequality_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_g_num,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_g_num_12_code_filter_iso_offset_upsample_valid.pkl")
            joblib.dump(self.data_g_value,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_g_value_12_code_filter_iso_offset_upsample_valid.pkl")
        #"""
        """
        if valid==False:
            path_data = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_mean_train_1_288.pkl'
            path_data_basic_group = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_basic_train_1_288.pkl'
            path_data_sd= '/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_sd_train_1_288.pkl'
            path_y_basic_group = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_basic_group_train_1_288.pkl'
            path_y_offset = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_offset_train_1_288.pkl'

        else:
            path_data = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_mean_valid_1_288.pkl'
            path_data_basic_group = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_basic_valid_1_288.pkl'
            path_data_sd = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/data_sd_valid_1_288.pkl'
            path_y_basic_group = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_basic_group_valid_1_288.pkl'
            path_y_offset = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/y_offset_valid_1_288.pkl'
        self.data_basic_group=joblib.load(path_data_basic_group) 
        self.data_sd_diff=joblib.load(path_data_sd) 
        self.data=joblib.load(path_data)    
        self.label_basic_group=joblib.load(path_y_basic_group)  
        self.label_offset=joblib.load(path_y_offset) 
        
        if valid==False:
            self.data = normalize_train(self.data)
            self.data_sd_diff = normalize_train(self.data_sd_diff,sd=True)
        else:
            self.data = normalize_test(self.data)
            self.data_sd_diff = normalize_test(self.data_sd_diff,sd=True)
        """
        print(self.data_basic_group.shape)
        print(self.data.shape)
        #self.label=y_train 
        self.len_data=self.data.shape[0]
        self.transform = transform
        #self.label = self.label.astype(int)  # numpy强制类型转换
        self.label_basic_group=torch.LongTensor(y_train_basic_group)
        self.label_offset=torch.LongTensor(y_train_offset)
        self.data=torch.FloatTensor(self.data)

        self.data =  self.data.unsqueeze(2)

    def __len__(self):
        return self.len_data
 
    def __getitem__(self, index):
        #data = torch.FloatTensor(data)
        return  torch.cat((self.data[index],self.data_basic_group[index]),dim=1),self.label_basic_group[index],self.label_offset[index]

class origin_dataset_test(Dataset):
    def __init__(self, transform,motif):
        super(origin_dataset_test, self).__init__()
        #"""
        path_test = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_12_code_filter_iso.rds'
        path_test_base_num_fraction = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_base_num_fraction_12_code_filter_iso.rds'
        path_test_std_diff = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_std_diff_12_code_filter_iso.rds'
        path_test_time_diff = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_time_diff_12_code_filter_iso.rds'
        path_test_basic_group = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_basic_group_12_code_filter_iso.rds'
        path_test_t_test_pval = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_t_test_pval_12_code_filter_iso.rds'
        path_test_u_test_pval = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_u_test_pval_12_code_filter_iso.rds'
        path_test_avg_basequality = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_avg_basequality_12_code_filter_iso.rds'
        path_test_g_num = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_g_num_12_code_filter_iso.rds'
        path_test_g_value = '/home/nipeng/chenzheng/data.basic_group/evalute_data_HP_g_value_12_code_filter_iso.rds'
        """
        path_test_basic_group = '/home/nipeng/chenzheng/data.basic_group/evalute_data_TP_basic_group_1_289.rds'
        path_test_sd_diff = '/home/nipeng/chenzheng/data.basic_group/evalute_data_TP_sd_diff_1_289.rds'
        path_test_nat = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_nat_1_289.pkl'
        path_test_wga = '/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_wga_1_289.pkl'
        path_test_mean_nat = '/home/nipeng/chenzheng/data.basic_group/evalute_data_TP_mean_nat_1_289.rds'
        path_test_mean_wga = '/home/nipeng/chenzheng/data.basic_group/evalute_data_TP_mean_wga_1_289.rds'
        path_test_sd_nat = '/home/nipeng/chenzheng/data.basic_group/evalute_data_TP_sd_nat_1_289.rds'
        path_test_sd_wga = '/home/nipeng/chenzheng/data.basic_group/evalute_data_TP_sd_wga_1_289.rds'
        """
        readRDS = robjects.r['readRDS']
        file_test = readRDS(path_test)
        file_test_base_num_fraction  = readRDS(path_test_base_num_fraction)
        file_test_std_diff  = readRDS(path_test_std_diff)
        file_test_time_diff  = readRDS(path_test_time_diff)
        file_test_basic_group  = readRDS(path_test_basic_group)
        file_test_t_test_pval  = readRDS(path_test_t_test_pval)
        file_test_u_test_pval  = readRDS(path_test_u_test_pval)
        file_test_avg_basequality  = readRDS(path_test_avg_basequality)
        file_test_g_num  = readRDS(path_test_g_num)
        file_test_g_value  = readRDS(path_test_g_value)
        """
        file_test_basic_group  = readRDS(path_test_basic_group)
        file_test_sd_diff  = readRDS(path_test_sd_diff)
        file_test_sd_nat  = readRDS(path_test_sd_nat)
        file_test_sd_wga  = readRDS(path_test_sd_wga)
        file_test_mean_nat  = readRDS(path_test_mean_nat)
        file_test_mean_wga  = readRDS(path_test_mean_wga)
        """
        with localconverter(ro.default_converter + pandas2ri.converter):
            pd_from_r_df_test = ro.conversion.rpy2py(file_test)
            pd_from_r_df_test_base_num_fraction = ro.conversion.rpy2py(file_test_base_num_fraction)
            pd_from_r_df_test_std_diff = ro.conversion.rpy2py(file_test_std_diff)
            pd_from_r_df_test_time_diff = ro.conversion.rpy2py(file_test_time_diff)
            pd_from_r_df_test_basic_group = ro.conversion.rpy2py(file_test_basic_group)
            pd_from_r_df_test_t_test_pval = ro.conversion.rpy2py(file_test_t_test_pval)
            pd_from_r_df_test_u_test_pval = ro.conversion.rpy2py(file_test_u_test_pval)
            pd_from_r_df_test_avg_basequality = ro.conversion.rpy2py(file_test_avg_basequality)
            pd_from_r_df_test_g_num = ro.conversion.rpy2py(file_test_g_num)
            pd_from_r_df_test_g_value = ro.conversion.rpy2py(file_test_g_value)
            """
            pd_from_r_df_test_basic_group = ro.conversion.rpy2py(file_test_basic_group)
            pd_from_r_df_test_sd_diff = ro.conversion.rpy2py(file_test_sd_diff)
            pd_from_r_df_test_sd_nat = ro.conversion.rpy2py(file_test_sd_nat)
            pd_from_r_df_test_sd_wga = ro.conversion.rpy2py(file_test_sd_wga)
            pd_from_r_df_test_mean_nat = ro.conversion.rpy2py(file_test_mean_nat)
            pd_from_r_df_test_mean_wga = ro.conversion.rpy2py(file_test_mean_wga)
            """
        print(np.any(pd_from_r_df_test_base_num_fraction.isnull()))
        #pd_from_r_df_test=pd_from_r_df_test.assign(row_number=range(len(pd_from_r_df_test)))
        """
        print(pd_from_r_df_test_basic_group)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', None)
        with open('output.txt', 'w') as f:
            # 使用print函数的file参数将输出打印到文件
            print("12_origin_filter_iso")
            print(pd_from_r_df_test_basic_group, file=f)
        
        #print(pd_from_r_df_test_basic_group)
        """
        joblib.dump(pd_from_r_df_test,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_base_num_fraction,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_base_num_fraction_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_std_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_std_diff_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_time_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_time_diff_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_basic_group_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_t_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_t_test_pval_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_u_test_pval,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_u_test_pval_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_avg_basequality,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_avg_basequality_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_g_num,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_g_num_12_code_filter_iso.pkl")
        joblib.dump(pd_from_r_df_test_g_value,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/HP_pd_from_r_df_test_g_value_12_code_filter_iso.pkl")
        
        self.label_basic_group=np.array(pd_from_r_df_test["label"])
        self.label_basic_group = np.where((self.label_basic_group=="CGCG_-9")|(self.label_basic_group=="CCTCC_-9")|
                                          (self.label_basic_group=="CTACT_-9")|(self.label_basic_group=="GGNCC_-9")|
                                          (self.label_basic_group=="RAACTC_-9"), 1, self.label_basic_group)
        self.label_basic_group = np.where((self.label_basic_group=="CAGAAA_-9")|(self.label_basic_group=="CCCRAG_-9")|
                                          (self.label_basic_group=="GATC_-9"), 2, self.label_basic_group)
        self.label_basic_group = self.label_basic_group.astype(np.int64)
        self.label_offset=np.array(pd_from_r_df_test["label"])
        self.label_offset = np.where((self.label_offset=="CGCG_-9")|(self.label_offset=="CAGAAA_-9")|
                                          (self.label_offset=="CTACT_-9")|(self.label_offset=="GATC_-9")|
                                          (self.label_offset=="GGNCC_-9")|(self.label_offset=="RAACTC_-9"), 3, self.label_offset)  
        self.label_offset = np.where((self.label_offset=="CCTCC_-9")|(self.label_offset=="CCCRAG_-9"), 4, self.label_offset) 
        self.label_offset = self.label_offset.astype(np.int64)

        #self.label_basic_group=np.array(self.label_basic_group,dtype=np.int64)   
        #self.label_offset=np.array(self.label_offset,dtype=np.int64)  
        """
        joblib.dump(pd_from_r_df_test,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_pd_from_r_df_test_1_289.pkl")
        joblib.dump(pd_from_r_df_test_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_pd_from_r_df_test_basic_group_1_289.pkl")
        joblib.dump(pd_from_r_df_test_sd_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_pd_from_r_df_test_sd_diff_1_289.pkl")
        joblib.dump(pd_from_r_df_test_mean_nat,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_pd_from_r_df_test_mean_nat_1_289.pkl")
        joblib.dump(pd_from_r_df_test_mean_wga,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_pd_from_r_df_test_mean_wga_1_289.pkl")
        joblib.dump(pd_from_r_df_test_sd_nat,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_pd_from_r_df_test_sd_nat_1_289.pkl")
        joblib.dump(pd_from_r_df_test_sd_wga,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_pd_from_r_df_test_sd_wga_1_289.pkl")
        """
        subet_test_data=pd_from_r_df_test[pd_from_r_df_test["label"]==motif]

        """        
        subet_test_data=pd_from_r_df_test
        subet_test_data_basic_group=pd_from_r_df_test_basic_group
        subet_test_data_sd_diff=pd_from_r_df_test_sd_diff
        subet_test_data_mean_nat=pd_from_r_df_test_mean_nat
        subet_test_data_mean_wga=pd_from_r_df_test_mean_wga
        subet_test_data_sd_nat=pd_from_r_df_test_sd_nat
        subet_test_data_sd_wga=pd_from_r_df_test_sd_wga
        """
        self.data=subet_test_data.values[:,1:13]
        self.data=self.data.astype(float) 
        #self.data = normalize_test(self.data)
        """
        self.data_basic_group=subet_test_data_basic_group.values[:,1:13]
        #embedding
        self.data_basic_group[self.data_basic_group=='A']=0
        self.data_basic_group[self.data_basic_group=='C']=1
        self.data_basic_group[self.data_basic_group=='T']=2
        self.data_basic_group[self.data_basic_group=='G']=3
        
        self.data_basic_group = self.data_basic_group.astype(int) 
        """
        #self.data_basic_group=np.eye(4)[self.data_basic_group] 
        self.len_data=self.data.shape[0]
        self.transform = transform

        """
        joblib.dump(self.data,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_mean_1_289.pkl")
        joblib.dump(self.data_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_basic_group_1_289.pkl")
        joblib.dump(self.data_sd_diff,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_sd_diff_1_289.pkl")
        joblib.dump(self.data_sd_nat,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_sd_nat_1_289.pkl")
        joblib.dump(self.data_sd_wga,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_sd_wga_1_289.pkl")
        joblib.dump(self.data_mean_nat,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_mean_nat_1_289.pkl")
        joblib.dump(self.data_mean_wga,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_mean_wga_1_289.pkl")
        joblib.dump(self.label_basic_group,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_y_basic_group_1_289.pkl")
        joblib.dump(self.label_offset,"/home/nipeng/chenzheng/mulan-methyl/data_pkl/TP_y_offset_1_289.pkl")
        """
        self.data=torch.FloatTensor(self.data)
        self.data_sd_diff=torch.FloatTensor(self.data_sd_diff)
        self.data_nat=torch.FloatTensor(self.data_nat)
        self.data_wga=torch.FloatTensor(self.data_wga)
        self.data_basic_group=torch.LongTensor(self.data_basic_group)
        self.data =  self.data.unsqueeze(2)
        self.data_sd_diff =  self.data_sd_diff.unsqueeze(2)
    def __len__(self):
        return self.len_data
 
    def __getitem__(self, index):
        #data = torch.FloatTensor(data)
        return  self.data[index],self.data_basic_group[index],self.data_sd_diff[index],self.data_nat[index],self.data_wga[index]
# 设置随机种子，让模型每次输出的结果都一样
seed_value = 42
random.seed(seed_value)                         # 设置 random 模块的随机种子
np.random.seed(seed_value)                      # 设置 numpy 模块的随机种子
torch.manual_seed(seed_value)                   # 设置 PyTorch 中 CPU 的随机种子
#tf.random.set_seed(seed_value)                 # 设置 Tensorflow 中随机种子
if torch.cuda.is_available():                   # 如果可以使用 CUDA，设置随机种子
    torch.cuda.manual_seed(seed_value)          # 设置 PyTorch 中 GPU 的随机种子
    torch.backends.cudnn.deterministic = True   # 使用确定性算法，使每次运行结果一样
    torch.backends.cudnn.benchmark = False      # 不使用自动寻找最优算法加速运算
# 检测GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#scaler = StandardScaler() 
dataset_train=origin_dataset_train(transform=None,valid=False)
dataset_valid=origin_dataset_train(transform=None,valid=True)
#dataset_test=origin_dataset_test(transform=None,motif="CGCG_-9")